In [1]:
import cv2 as cv

## Support functions

In [2]:
def createTrackerByName(tracker_type:str):
    """
    Returns the tracker object based on its name.
    """
    TRACKER_TYPES = ['BOOSTING', 'MIL', 'KCF', 'TLD', 'MEDIANFLOW', 'MOSSE', 'CSRT']
    
    if tracker_type == TRACKER_TYPES[0]:
        tracker = cv.legacy.TrackerBoosting_create()
    elif tracker_type == TRACKER_TYPES[1]:
        tracker = cv.legacy.TrackerMIL_create()
    elif tracker_type == TRACKER_TYPES[2]:
        tracker = cv.legacy.TrackerKCF_create()
    elif tracker_type == TRACKER_TYPES[3]:
        tracker = cv.legacy.TrackerTLD_create()
    elif tracker_type == TRACKER_TYPES[4]:
        tracker = cv.legacy.TrackerMedianFlow_create()
    elif tracker_type == TRACKER_TYPES[5]:
        tracker = cv.legacy.TrackerMOSSE_create()
    elif tracker_type == TRACKER_TYPES[6]:
        tracker = cv.legacy.TrackerCSRT_create()
    else:
        tracker = None
        raise ValueError(f"[ERROR] Invalid selection! Available tracker: {TRACKER_TYPES}")
        
    return tracker

def drawInfoBox(image, upperLeftCorner:tuple, lowerRightCorner:tuple, status:str, fps:int) -> None:
    """
    Displays information about the tracker and the frame rate.
    """
    scale_text = 0.7
    font_face = cv.FONT_HERSHEY_SIMPLEX
    
    colors_status = {
        "Tracking": (0, 255, 0),
        "Not tracking": (0, 0, 255),
        "Selection": (255, 0, 0)
    }
    
    cv.rectangle(image, upperLeftCorner, lowerRightCorner, (255, 255, 0), 2, 1)
    
    fps_x_pos, fps_y_pos = upperLeftCorner[0] + 10, upperLeftCorner[1] + 30
    cv.putText(image, "FPS: ", (fps_x_pos, fps_y_pos), font_face, scale_text, (0, 255, 255), 2)
    cv.putText(image, str(fps), (fps_x_pos + 55, fps_y_pos), font_face, scale_text, (255, 0, 0), 2)
    
    status_x_pos, status_y_pos = upperLeftCorner[0] + 10, upperLeftCorner[1] + 55
    cv.putText(image, "Status: ", (status_x_pos, status_y_pos), font_face, scale_text, (0, 255, 255), 2)
    cv.putText(image, status, (status_x_pos + 80, status_y_pos), font_face, scale_text, colors_status[status], 2)

def drawBBox(image, bbox:tuple) -> None:
    """
    Draws a rectangle on the image according to the specified coordinates 
    of the upper-left corner, width and height.
    """
    x, y, w, h = [int(el) for el in bbox]
    cv.rectangle(image, (x, y), ((x+w), (y+h)), (0, 255, 0), 3, 1)
    
def selectObjects(image, name_window:str) -> list:
    """
    Selecting an objects for tracking. 
    Returns the coordinates of the bounding boxes.
    """
    bboxes = []
    
    while True:
        if bboxes:
            for bbox in bboxes:
                drawBBox(image, bbox)
        
        bbox = cv.selectROI(name_window, image, False)
        
        if checkValidBBox(bbox):
            bboxes.append(bbox)
        
        key = cv.waitKey(500) & 0xff
        if key == ord('e'):
            break
            
    return bboxes

def checkValidBBox(bbox):
    """
    Checking that the bbox is valid and can be displayed.
    True - Valid, False - Invalid.
    """
    return not all([el == 0 for el in bbox])

## Main

In [3]:
name_window = "Tracking"
status = "Not tracking"

tracker_exists = False

# Сapturing an image from a webcam
capture = cv.VideoCapture(0)

# Choosing a tracker
name_tracker = "KCF"
multi_tracker = None

while True:
    timer = cv.getTickCount()
    success, image = capture.read()
    
    ## Tracking
    if tracker_exists:
        _, bboxes = multi_tracker.update(image)
        
        is_tracking = False
        for bbox in bboxes:
            if checkValidBBox(bbox):
                drawBBox(image, bbox)
                is_tracking = True
                
        if is_tracking:         
            status = "Tracking"    
        else:
            status = "Not tracking"
    else:
        status = "Not tracking"
    
    ## FPS measurement
    fps = cv.getTickFrequency() / (cv.getTickCount() - timer)
    
    ## Information and image output
    drawInfoBox(image, (10, 10), (250, 80), status, int(fps))
    cv.imshow(name_window, image)
    
    ## Reading the pressed key
    key = cv.waitKey(1) & 0xFf
    
    if key == ord('s'): # Pressing the S key, the objects to be tracked are selected
        status = "Selection"
        _, image = capture.read()
        drawInfoBox(image, (10, 10), (250, 80), status, int(fps))
        
        for bbox in selectObjects(image, name_window):
            if multi_tracker is None:
                multi_tracker = cv.legacy.MultiTracker_create()
                tracker_exists = True
            
            multi_tracker.add(createTrackerByName(name_tracker.upper()), image, bbox)
                
    elif key == ord('c'): # Pressing the C key, the history of objects for tracking is cleared
        multi_tracker = None
        tracker_exists = False
        status = "Not tracking"
        
    elif key == ord('q'): # Pressing the Q key exits the program
        cv.destroyAllWindows()
        break